In [1]:
%load_ext tensorboard

In [ ]:
%%shell

if [[ ! -d "/content/8th-sem-project" ]]; then
    git clone https://github.com/firekind/8th-sem-project --recurse-submodules &> /dev/null
    echo "Cloned repo."
else
    cd /content/8th-sem-project && git pull
    echo "Pulled repo."
fi

pip uninstall -y torchtext
pip install -r "/content/8th-sem-project/requirements.txt"
echo "Downloaded dependencies."

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%shell
DATA_DIR=/content/8th-sem-project/data
if [[ ! -d $DATA_DIR ]]; then
    mkdir $DATA_DIR
    echo "extracting dataset..."
    tar -xvf "/content/drive/My Drive/8th-sem-project/yolo-mask.tar.xz" -C $DATA_DIR &> /dev/null
    echo "done."
else
    echo "dataset already extracted."
fi

extracting dataset...
done.


In [5]:
%%shell
cd /content/8th-sem-project && sh ./download-models.sh

yolov3.weights      100%[===================>] 236.52M  7.16MB/s    in 18s     


In [6]:
import os
os.chdir("/content/8th-sem-project")

In [7]:
import torch
from src.dataset import dataset, collate_fn
from src.config import Config
from src.trainer import Model

from athena.utils.progbar import ProgbarCallback
import pytorch_lightning as pl

In [8]:
config = Config(
    DATA_DIR="data/data",
    IMG_SIZE=640,
    MIN_IMG_SIZE=320,
    BATCH_SIZE=10,
    MIDAS_LOSS_WEIGHT=0,
    YOLO_LOSS_WEIGHT=1,
    EPOCHS=10
)
train_dataset = dataset(config)
loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

test_dataset = dataset(config, train=False)
loader_test = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    collate_fn=collate_fn
)

Caching labels data/data/labels.npy (3161 found, 0 missing, 42 empty, 0 duplicate, for 3203 images): 100%|██████████| 3203/3203 [00:00<00:00, 5461.80it/s]
Caching labels data/data/labels.npy (311 found, 0 missing, 7 empty, 0 duplicate, for 318 images): 100%|██████████| 318/318 [00:00<00:00, 4534.51it/s]


In [9]:
model = Model(config, len(loader), 4, train_dataset.yolo_labels)

Model Summary: 222 layers, 6.15399e+07 parameters, 6.15399e+07 gradients


In [10]:
log_dir = "/content/drive/MyDrive/8th-sem-project"
name = "Yolo"
tensorboard_logger = pl.loggers.TensorBoardLogger(
    log_dir, name="", version=name, default_hp_metric=False
)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(
        log_dir, 
        name,
        "checkpoints",
        "last",
    ),
)

In [11]:
pl.seed_everything(0)
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    gpus=1,
    logger=tensorboard_logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[ProgbarCallback()],
    progress_bar_refresh_rate=20,
    automatic_optimization=False
)

In [12]:
trainer.fit(
    model,
    loader,
    loader_test
)

Epoch: 1 / 10
321/321 [==============================] - 942s 3s/step - yolo loss: 0.8531
Validation set: avg yolo val loss: 7.4801, yolo mAP: 0.3766, avg yolo loss: 1.7079

Epoch: 2 / 10
321/321 [==============================] - 933s 3s/step - yolo loss: 0.7982
Validation set: avg yolo val loss: 6.7621, yolo mAP: 0.4352, avg yolo loss: 0.9925

Epoch: 3 / 10
321/321 [==============================] - 928s 3s/step - yolo loss: 0.9052
Validation set: avg yolo val loss: 6.4508, yolo mAP: 0.4801, avg yolo loss: 0.8482

Epoch: 4 / 10
321/321 [==============================] - 930s 3s/step - yolo loss: 0.7267
Validation set: avg yolo val loss: 5.8824, yolo mAP: 0.5130, avg yolo loss: 0.7610

Epoch: 5 / 10
321/321 [==============================] - 933s 3s/step - yolo loss: 0.9551
Validation set: avg yolo val loss: 5.8047, yolo mAP: 0.5164, avg yolo loss: 0.7027

Epoch: 6 / 10
321/321 [==============================] - 923s 3s/step - yolo loss: 0.5482
Validation set: avg yolo val loss: 5.789

1

In [15]:
from src.yolov3.models import save_weights
save_weights(model.yolo, path="/content/drive/MyDrive/8th-sem-project/yolo-face.weights")